In [40]:
import json
import pandas as pd
from collections import Counter, defaultdict

# Exploratory Data Analysis

- Time range of data
- Common cities, states, and zip codes 
- Common cuisines overall and per CBSA

CBSA, [Wikipedia](https://en.wikipedia.org/wiki/Core-based_statistical_area)
> core-based statistical area (CBSA) is a U.S. geographic area defined by the Office of Management and Budget (OMB) that consists of one or more counties (or equivalents) anchored by an urban center of at least 10,000 people plus adjacent counties that are socioeconomically tied to the urban center by commuting

https://www.huduser.gov/portal/datasets/usps_crosswalk.html

Ideas: 

Characterization of food: 

- Authenticity
- "axes of evaluation": adjectives used to describe foods
    - sweetness (e.g. are east asian people more favoring less sweet, southerners favoring more sweet)
    - spiciness (e.g. are white people more critical of spicy)
- reviews involving "support" and slack/activism 
- specificity of regions / cuisines within reviews: "South Indian" vs. "Indian", "Chinese" vs. "Sichuan", "African" vs. "Ethipian" 
- how semantically distinct are people's desecriptions within categories of food
- portion size (ratio of blank plate to food on the plate)

Differential axes: 
- Geography
- Urban vs. rural 
- Pairings of cuisines: Soul Food vs. Southern, American vs. Mexican (time and border wall politics), Italian vs. Mexican
- Issue/event-based reviewing, Pakistani restaurant getting burned down in Minneapolis, did that change anything? 
- before and after receiving awards 
- immigration and establishment of cuisine times 
- demographics of zip code 
- college towns vs other towns

Get entities (whether certain dishes are called out in different regions) - orange chicken in midwest and south vs. something w/ Sichuan peppercorns in other regions 

Food trends: Sichuan was popular at some point and now there are Chinese restaurants that cater towards Sichuan

In [4]:
df = pd.read_json('yelp_academic_dataset_business.json', lines=True)
df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,AB,T6J 5H2,53.468419,-113.492054,3.0,13,1,"{'ByAppointmentOnly': 'False', 'RestaurantsPri...","Nail Salons, Beauty & Spas","{'Monday': '10:0-19:30', 'Tuesday': '10:0-19:3..."
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,TN,37204,36.115118,-86.766925,4.0,5,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Pets, Nurseries & Gardening, Pet Stores, Hobby...","{'Monday': '9:30-17:30', 'Tuesday': '9:30-17:3..."
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,IN,46250,39.908707,-86.065088,3.5,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Shopping, Jewelry, Piercing, Toy Stores, Beaut...",None
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,IL,62025,38.782351,-89.950558,4.0,24,1,"{'BusinessParking': '{'garage': False, 'street...","Fitness/Exercise Equipment, Eyewear & Optician...","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ..."


In [5]:
df.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')

In [14]:
df = df[['business_id', 'city', 'state', 'postal_code', 'stars', 'review_count', 'categories']]
df = df[df['categories'].notna()]
df = df[df['categories'].str.contains("Restaurants")]
df['city_state'] = df['city'] + '_' + df['state']
df

,business_id,city,state,postal_code,stars,review_count,categories,city_state
3,MTSW4McQd7CbVtyjqoe9mw,Philadelphia,PA,19107,4.0,80,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",Philadelphia_PA
5,CF33F8-E6oudUQ46HnavjQ,Ashland City,TN,37015,2.0,6,"Burgers, Fast Food, Sandwiches, Food, Ice Crea...",Ashland City_TN
8,k0hlBqXX-Bt0vf1op7Jr1w,Affton,MO,63123,3.0,19,"Pubs, Restaurants, Italian, Bars, American (Tr...",Affton_MO
9,bBDDEgkFA1Otx9Lfe7BZUQ,Nashville,TN,37207,1.5,10,"Ice Cream & Frozen Yogurt, Fast Food, Burgers,...",Nashville_TN
11,eEOYSgkmpB90uNA7lDOMRA,Tampa Bay,FL,33602,4.0,10,"Vietnamese, Food, Restaurants, Food Trucks",Tampa Bay_FL
...,...,...,...,...,...,...,...,...
150325,l9eLGG9ZKpLJzboZq-9LRQ,Clifton Heights,PA,19018,3.0,11,"Restaurants, Sandwiches, Convenience Stores, C...",Clifton Heights_PA
150327,cM6V90ExQD6KMSU3rRB5ZA,Boise,ID,83704,4.0,33,"Cafes, Juice Bars & Smoothies, Coffee & Tea, R...",Boise_ID
150336,WnT9NIzQgLlILjPT0kEcsQ,Philadelphia,PA,19147,4.5,35,"Restaurants, Mexican",Philadelphia_PA
150339,2O2K6SXPWv56amqxCECd4w,Aston,DE,19014,4.5,14,"Restaurants, Comfort Food, Food, Food Trucks, ...",Aston_DE


In [21]:
city_counts = Counter(df.city_state.value_counts().to_dict())
print(city_counts.most_common(40))

[('Philadelphia_PA', 5852), ('Tampa_FL', 2960), ('Indianapolis_IN', 2862), ('Nashville_TN', 2502), ('Tucson_AZ', 2466), ('New Orleans_LA', 2259), ('Edmonton_AB', 2166), ('Saint Louis_MO', 1790), ('Reno_NV', 1286), ('Boise_ID', 847), ('Santa Barbara_CA', 767), ('Clearwater_FL', 678), ('Wilmington_DE', 619), ('St. Louis_MO', 542), ('Metairie_LA', 522), ('Saint Petersburg_FL', 491), ('Franklin_TN', 442), ('St. Petersburg_FL', 404), ('Sparks_NV', 334), ('Brandon_FL', 326), ('Meridian_ID', 312), ('Largo_FL', 310), ('Cherry Hill_NJ', 297), ('Carmel_IN', 291), ('West Chester_PA', 280), ('Kenner_LA', 228), ('New Port Richey_FL', 226), ('Goleta_CA', 220), ('Greenwood_IN', 207), ('Palm Harbor_FL', 207), ('Fishers_IN', 203), ('Riverview_FL', 198), ('Trenton_NJ', 192), ('Dunedin_FL', 188), ('Hendersonville_TN', 183), ('Lutz_FL', 178), ('Smyrna_TN', 167), ('Pinellas Park_FL', 167), ('Bensalem_PA', 165), ('Florissant_MO', 165)]


In [27]:
categories = df.categories.to_list()
category_counts = Counter()
for cat in categories: 
    category_counts.update(cat.split(', '))
print(category_counts.most_common(100))

[('Restaurants', 52268), ('Food', 15472), ('Nightlife', 8723), ('Sandwiches', 8366), ('Bars', 8337), ('American (Traditional)', 8139), ('Pizza', 7093), ('Fast Food', 6472), ('Breakfast & Brunch', 6239), ('American (New)', 6097), ('Burgers', 5636), ('Mexican', 4600), ('Italian', 4573), ('Coffee & Tea', 4053), ('Seafood', 3539), ('Chinese', 3169), ('Event Planning & Services', 3068), ('Salad', 3064), ('Chicken Wings', 2966), ('Cafes', 2756), ('Delis', 2393), ('Caterers', 2059), ('Specialty Food', 2011), ('Bakeries', 1889), ('Desserts', 1838), ('Japanese', 1830), ('Sports Bars', 1797), ('Sushi Bars', 1717), ('Barbeque', 1694), ('Asian Fusion', 1547), ('Steakhouses', 1506), ('Diners', 1494), ('Cocktail Bars', 1405), ('Pubs', 1397), ('Mediterranean', 1263), ('Vegetarian', 1158), ('Beer', 1154), ('Wine & Spirits', 1154), ('Ice Cream & Frozen Yogurt', 1088), ('Arts & Entertainment', 1086), ('Soup', 1061), ('Tacos', 1020), ('Juice Bars & Smoothies', 1015), ('Food Trucks', 1008), ('Southern', 9

In [33]:
categories_of_interest = set(['American (Traditional)', 'American (New)', 'Mexican', 'Italian', 'Chinese', 
                         'Japanese', 'Asian Fusion', 'Mediterranean', 'Southern', 'Thai', 'Cajun/Creole', 
                         'Tex-Mex', 'Vietnamese', 'Indian', 'Latin American', 'Greek', 
                         'Caribbean', 'Middle Eastern', 'Soul Food', 'Ethnic Food', 'French', 'Korean', 
                         'Halal', 'Spanish', 'Cuban', 'Canadian (New)', 'Pakistani', 'Ramen'])

**TODO**: discuss what to do about things like "Mediterranean" and "Middle Eastern"

In [42]:
country_of_origin = { 
    # there are some cuisines that aren't countries, this is only for countries
    'Mexican': 'Mexico', 
    'Italian': 'Italy', 
    'Chinese': 'China', 
    'Japanese': 'Japan', 
    'Thai': 'Thailand',
    'Vietnamese': 'Vietnam', 
    'Indian': 'India', 
    'Greek': 'Greece', 
    'French': 'France',
    'Korean': 'Korea', 
    'Spanish': 'Spain', 
    'Cuban': 'Cuba', 
    'Canadian (New)': 'Canada', 
    'Pakistani': 'Pakistan', 
}

In [34]:
business_categories = df[['business_id', 'categories']]
business_categories = business_categories.set_index('business_id').T.to_dict()
businesses_to_keep = set()
for business in business_categories: 
    cats = business_categories[business]['categories']
    if set(cats.split(', ')) & categories_of_interest: 
        businesses_to_keep.add(business)

In [49]:
with open('businesses_to_keep.txt', 'w') as outfile: 
    for business_id in businesses_to_keep: 
        outfile.write(business_id + '\n')

In [48]:
business_regions = defaultdict(list)
for business in business_categories: 
    cats = business_categories[business]['categories']
    cats = set(cats.split(', '))
    for cat in cats: 
        if cat in country_of_origin: 
            country = country_of_origin[cat]
            business_regions[business].append(country)
with open('business_regions.json', 'w') as outfile: 
    json.dump(business_regions, outfile)

In [39]:
print("Businesses with regional cuisine labels:", len(businesses_to_keep))
cuisine_df = df[df['business_id'].isin(businesses_to_keep)]
cuisine_df

Businesses with regional cuisine labels: 31961


,business_id,city,state,postal_code,stars,review_count,categories,city_state
8,k0hlBqXX-Bt0vf1op7Jr1w,Affton,MO,63123,3.0,19,"Pubs, Restaurants, Italian, Bars, American (Tr...",Affton_MO
11,eEOYSgkmpB90uNA7lDOMRA,Tampa Bay,FL,33602,4.0,10,"Vietnamese, Food, Restaurants, Food Trucks",Tampa Bay_FL
12,il_Ro8jwPlHresjw9EGmBg,Indianapolis,IN,46227,2.5,28,"American (Traditional), Restaurants, Diners, B...",Indianapolis_IN
14,0bPLkL0QhhPO5kt1_EXmNQ,Largo,FL,33771,4.5,100,"Food, Delis, Italian, Bakeries, Restaurants",Largo_FL
15,MUTTqe8uqyMdBl186RmNeA,Philadelphia,PA,19106,4.0,245,"Sushi Bars, Restaurants, Japanese",Philadelphia_PA
...,...,...,...,...,...,...,...,...
150306,wVxXRFf10zTTAs11nr4xeA,Philadelphia,PA,19128,3.0,55,"Restaurants, Specialty Food, Food, Sandwiches,...",Philadelphia_PA
150313,sf_oQ62L8UEnOOLf00nNGA,Hermitage,TN,37076,3.0,6,"Restaurants, Pizza, Fast Food, Chicken Wings, ...",Hermitage_TN
150319,8n93L-ilMAsvwUatarykSg,Philadelphia,PA,19104,3.0,22,"Coffee & Tea, Food, Sandwiches, American (Trad...",Philadelphia_PA
150336,WnT9NIzQgLlILjPT0kEcsQ,Philadelphia,PA,19147,4.5,35,"Restaurants, Mexican",Philadelphia_PA


# Authenticity analysis

Replicate https://ny.eater.com/2019/1/18/18183973/authenticity-yelp-reviews-white-supremacy-trap

- Count proportion of reviews that mention at least one of the following: "authentic", "authenticity", "legitimate", "traditional", "Americanized", or country of cuisine origin
    - Categorize by cuisine 
    - Categorize by metropolitan area
    
- Calculate correlation b/t mentions of authenticity and average star rating
    - Categorize by cuisine